In [ ]:
from pathlib import Path
import os

import rasterio
import shapely

from planet_processing import PlanetOrderDownloader as PoD, PlanetVisualizer as Pv

REQUEST_ID = 0
AOI = 'POLYGON ((29.912338 51.192255, 29.912338 51.481811, 30.459595 51.481811, 30.459595 51.192255, 29.912338 51.192255))'
START_DATE = '2021-12-26'
END_DATE = '2022-04-29'
ORDERID = '0753a164-3775-4c47-b4e6-b8e38e9d2295'
PLANET_API_KEY = 'PLAK8b275ed2feab4db49e90ca77af81c060'

In [ ]:
NAME = "Planet Downloader"
NB_USER = os.getenv('NB_USER')
#DATA_DIR = Path(f"/home/{NB_USER}/work")
DATA_DIR = Path(f"/home/quantum/result")

API_KEY_DIR = DATA_DIR / ".secret/planet_api_key.json"
LOAD_DIR = DATA_DIR / "satellite_imagery"
RESULTS_DIR = DATA_DIR / "results/planet_downloader" / str(REQUEST_ID)
TEMP_DATA_DIR = DATA_DIR / 'temp'

TEMP_DIR = TEMP_DATA_DIR / ORDERID
TEMP_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
polygon_aoi = shapely.wkt.loads(AOI)

In [ ]:
downloader = PoD(PLANET_API_KEY, LOAD_DIR, polygon_aoi)
downloader.set_order_id(ORDERID)
downloader.poll_for_success()
order_name, order_archive_name = downloader.get_order_info()
downloader.download_order_archive()

In [ ]:
archive_path = LOAD_DIR / ORDERID / order_archive_name

In [ ]:
visualizer = Pv(archive_path, TEMP_DIR, RESULTS_DIR, delete_temp=True)
visualizer.set_max_timeout_for_img_generation(2000)
output_path = visualizer.run()

In [ ]:
with rasterio.open(output_path, "r+") as dst:
        dst.update_tags(start_date=START_DATE, end_date=END_DATE, request_id=REQUEST_ID, name=Path(output_path).stem)
       